In [5]:
import torch 
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import time

In [6]:
transform = transforms.Compose([transforms.Resize((100,100)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

In [9]:
size_row	= 100    # height of the image
size_col  	= 100   # width of the image

train_num=1027
test_num=256
batch=3
#train_data_path = 'relative path of training data set'
train_data_path = './horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch, shuffle=False, num_workers=1) 

In [10]:
validation_data_path = './horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=3, shuffle=False, num_workers=1)  

## Use the logistic funtion

$σ(z)={\frac{1}{1+exp(-z))}}$

In [11]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# make cluster

In [12]:
def Loss_Func(label,sigm,num):
    return (1/num)*np.sum(Funtion(label,sigm))

In [13]:
def Funtion(label,sigm):
    return -((label)*np.log(sigm)+(1-label)*np.log(1-sigm))

In [14]:
def Accuracy(label,sigm,num):
#     print(len(sigm),"len(sigm)")
#     count =0
#     for i in range(len(sigm)):
#         if(sigm[i]>0.5):
#             sigm[i]=1
#             if(sigm[i]-label[i]==0):
#                 count=count+1
            
#         else:
#             sigm[i]=0
#             if(sigm[i]-label[i]==0):
#                 count=count+1
#     return count / batch   

#     cor = 0
#     for i in range(num):
#         if (sigm[i] > 0.5 and label[i] == 1):
#             cor = cor + 1
#         elif (sigm[i] <= 0.5 and label[i] == 0):
#             cor = cor + 1
#     return cor / num

    sigm=sigm.reshape(-1)        
    arr=np.array(list(map(lambda x: 1 if x>=0.5 else 0, sigm)))
    arrs = list(filter(lambda x: x == 0, arr - label))
    return len(arrs) / num
   


# make H function
$y_{i}=σ(z_{i})$ where $z_{i}=w^{T}x_{i}+b$

In [15]:
def M(w,data):
    return np.dot(w,data)

# Calculate the derivative
## ${\sigma L\over \sigma u_{i}}$ =  $\sum_{i=1}^{N}x(\sigma(c)-h){(\sigma(b))(1-\sigma(b))}(\sigma(a))(1-\sigma(a))$

In [16]:
#a,b,c,v,w,train_matrix,
def d_sigmoid(m):
    return sigmoid(m)*(1-sigmoid(m))

In [17]:
#a,b,c,v,w,train_matrix,
def d_u(a,b,c,x,v,w,label):
#     temp1=sigmoid(c)-label
#     temp2=w*d_sigmoid(b)
#     temp3=v*d_sigmoid(a)
#     temp4=x
#     temp=np.dot(np.dot(np.dot(temp1,temp2),temp3),temp4.T)
    temp=sigmoid(c)-label
    temp=np.dot(w.T,temp)
    temp=temp*d_sigmoid(b)
    temp=np.dot(v.T,temp)
    temp=temp*d_sigmoid(a)
    temp=np.dot(temp,x.T)
    return (1/len(a[0]))* temp

In [18]:
#c,b,a,train_matrix,v,w
def d_v(a,b,c,w,label):

    temp=(sigmoid(c)-label)
    temp=np.dot(w.T,temp)
    temp=temp*d_sigmoid(b)
    temp=np.dot(temp,sigmoid(a).T)
    return (1/len(b[0]))* temp

In [19]:
#a,b,c,v,w,train_matrix,
def d_w(b,c,label):
    temp=np.dot((sigmoid(c)-label),sigmoid(b).T)
    return (1/len(c[0]))*temp

## Make the  Train_matrix

In [20]:
train_matrix=np.zeros(((size_col*size_row+1),train_num),dtype=float)
train_matrix_count=0
train_label=np.zeros(train_num,dtype=float)

NUM_EPOCH=1
for epoch in range (NUM_EPOCH):
    # load training images of the batch size for every iteration
    for i, data in enumerate(trainloader):

        # inputs is the image
        # labels is the class of the image
        inputs, labels = data
        
        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        inputs=np.array(inputs)
        labels=np.array(labels)
        
        for i in range(len(inputs)):
            for j in range((size_col*size_row)):
                train_matrix[j][train_matrix_count+i]=inputs[i,0,int(j/100),int(j%100)]
            train_matrix[(size_col*size_row)][train_matrix_count+i]=1
            train_label[train_matrix_count+i]=labels[i]
        train_matrix_count=train_matrix_count+len(inputs)
print(train_matrix)

[[0.59607846 0.60000002 0.6156863  ... 0.35686275 0.35294119 0.35294119]
 [0.59607846 0.59607846 0.6156863  ... 0.35686275 0.35294119 0.34901962]
 [0.59607846 0.60000002 0.6156863  ... 0.35294119 0.34901962 0.34901962]
 ...
 [0.90196079 0.79215688 0.66274512 ... 0.4627451  0.39607844 0.41568628]
 [0.87450981 0.75294119 0.69803923 ... 0.23529412 0.3882353  0.49411765]
 [1.         1.         1.         ... 1.         1.         1.        ]]


## Make the Testing Set

In [21]:
test_matrix=np.zeros(((size_col*size_row+1),test_num),dtype=float)
test_matrix_count=0
test_label=np.zeros(test_num,dtype=float)

NUM_EPOCH=1
for epoch in range (NUM_EPOCH):
    # load testing images of the batch size for every iteration
    for i, data in enumerate(valloader):

        # inputs is the image
        # labels is the class of the image
        inputs, labels = data
        
        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        inputs=np.array(inputs)
        labels=np.array(labels)
        
        for i in range(len(inputs)):
            for j in range((size_col*size_row)):
                test_matrix[j][test_matrix_count+i]=inputs[i,0,int(j/100),int(j%100)]
            test_matrix[(size_col*size_row)][test_matrix_count+i]=1
            test_label[test_matrix_count+i]=labels[i]
        test_matrix_count=test_matrix_count+len(inputs)





In [ ]:
train_varLoss=[]
train_varAccuracy=[]

test_varLoss=[]
test_varAccuracy=[]

times=[]
#make 2 hidden layer with k1 , k2 
k1=10
k2=5

#make hidden layer's matrix
u=np.array(np.random.randn(k1,10001))
v=np.array(np.random.randn(k2,k1))
w=np.array(np.random.randn(1,k2))

#determine the leaning rate
p=0.05

past=0
while_count=0
while True:
    #변수들과의 곱
    start = time.time()
    
    a_train=M(u,train_matrix)
    b_train=M(v,a_train)
    c_train=M(w,b_train)
    

    a_test=M(u,test_matrix)
    b_test=M(v,a_test)
    c_test=M(w,b_test)
    
    Der_w=d_w(b_train,c_train,train_label)
    Der_v=d_v(a_train,b_train,c_train,w,train_label)
    Der_u=d_u(a_train,b_train,c_train,train_matrix,v,w,train_label)

    w=w-p*Der_w
    v=v-p*Der_v
    u=u-p*Der_u
   
    end= time.time()
    
    train_Loss=Loss_Func(train_label,sigmoid(c_train),train_num)
    train_varLoss.append(train_Loss)
    print(train_Loss)
    test_Loss=Loss_Func(test_label,sigmoid(c_test),test_num)
    test_varLoss.append(test_Loss)
    
        
    train_Accu=Accuracy(train_label,sigmoid(c_train),train_num)
    train_varAccuracy.append(train_Accu)
    test_Accu=Accuracy(test_label,sigmoid(c_test),test_num)
    test_varAccuracy.append(test_Accu)
    
    times.append(end-start)
    
    while_count=while_count+1
        
    if abs(past - train_Loss)< 10e-6 and while_count>20:
        print("finish")
        break
    else:
        past = train_Loss
        continue

C:\Users\mose\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
C:\Users\mose\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in multiply
  


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


C:\Users\mose\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars


inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
3.3571930351903743
3.35007716477

1.1909467425111526
1.0951031309825665
1.1909455783592888
1.0951026501899959
1.1909444214269256
1.0951021767049232
1.1909432715100197
1.0951017103192657
1.1909421284102353
1.095101250830839
1.1909409919347738
1.095100798043192
1.1909398618962235
1.0951003517654407
1.190938738112416
1.0950999118121039
1.1909376204062672
1.0950994780029468
1.1909365086056418
1.0950990501628395
1.1909354025432188
1.0950986281215989
1.1909343020563476
1.0950982117138526
1.1909332069869245
1.0950978007789027
1.1909321171812641
1.0950973951605858
1.190931032489973
1.0950969947071507
1.1909299527678359
1.0950965992711315
1.1909288778736946
1.095096208709217
1.1909278076703342
1.0950958228821446
1.1909267420243739
1.095095441654575
1.1909256808061612
1.0950950648949915
1.1909246238896705
1.0950946924755758
1.1909235711523902
1.0950943242721216
1.1909225224752402
1.0950939601639156
1.1909214777424653
1.0950936000336515
1.1909204368415465
1.0950932437673229
1.1909193996631071
1.09509289125414
1.1909183661008336
1

...

1.1711147229063625
1.0921625673697353
1.1711133328656285
1.0921624278211626
1.1711119428088457
1.092162288261779
1.1711105527360104
1.0921621486915891
1.1711091626471126
1.0921620091106006
1.1711077725421482
1.0921618695188156
1.1711063824211099
1.0921617299162425
1.1711049922839916
1.092161590302886
1.1711036021307866
1.0921614506787511
1.1711022119614884
1.0921613110438442
1.1711008217760914
1.0921611713981687
1.171099431574589
1.0921610317417318
1.171098041356974
1.0921608920745387
1.171096651123241
1.0921607523965946
1.1710952608733833
1.0921606127079049
1.1710938706073941
1.092160473008476
1.171092480325267
1.0921603332983116
1.171091090026997
1.0921601935774188
1.1710896997125748
1.092160053845802
1.1710883093819981
1.0921599141034686
1.1710869190352575
1.0921597743504199
1.1710855286723463
1.0921596345866669
1.1710841382932597
1.0921594948122106
1.1710827478979908
1.0921593550270605
1.1710813574865349
1.0921592152312185
1.1710799670588834
1.092159075424691
1.1710785766150298
1.0

1.0921007791483668
1.1705056574476698
1.0921006354526954
1.1705042597797761
1.0921004917489285
1.1705028620930022
1.0921003480370721
1.1705014643873408
1.0921002043171344
1.1705000666627865
1.09210006058912
1.170498668919332
1.092099916853039
1.170497271156972
1.0920997731088966
1.1704958733756978
1.0920996293566994
1.170494475575505
1.0920994855964545
1.1704930777563853
1.0920993418281695
1.170491679918332
1.0920991980518515
1.1704902820613405
1.0920990542675075
1.1704888841854022
1.0920989104751422
1.1704874862905112
1.0920987666747652
1.1704860883766601
1.092098622866384
1.170484690443845
1.0920984790500035
1.1704832924920567
1.0920983352256315
1.1704818945212894
1.0920981913932741
1.1704804965315363
1.0920980475529416
1.170479098522792
1.0920979037046366
1.170477700495048
1.0920977598483685
1.1704763024482985
1.092097615984144
1.1704749043825378
1.0920974721119694
1.1704735062977578
1.0920973282318536
1.1704721081939538
1.0920971843438017
1.1704707100711176
1.09209704044782
1.17046

1.092037459444197
1.1698943927270902
1.092037312914135
1.1698929861958272
1.0920371663792507
1.1698915796427654
1.0920370198395517
1.1698901730678937
1.092036873295049
1.1698887664712097
1.0920367267457467
1.1698873598527018
1.0920365801916554
1.1698859532123673
1.0920364336327821
1.1698845465501968
1.0920362870691354
1.169883139866186
1.092036140500722
1.1698817331603253
1.092035993927552
1.1698803264326096
1.0920358473496332
1.1698789196830326
1.0920357007669712
1.1698775129115855
1.0920355541795757
1.1698761061182625
1.0920354075874554
1.1698746993030582
1.0920352609906177
1.1698732924659634
1.0920351143890694
1.1698718856069723
1.092034967782821
1.169870478726078
1.0920348211718782
1.169869071823275
1.09203467455625
1.1698676648985535
1.0920345279359436
1.1698662579519075
1.0920343813109679
1.1698648509833327
1.0920342346813312
1.169863443992821
1.0920340880470416
1.1698620369803643
1.0920339414081075
1.1698606299459573
1.0920337947645342
1.1698592228895917
1.0920336481163329
1.169

1.0919732274187397
1.1692789834117774
1.0919730795681983
1.1692775667215392
1.0919729317164757
1.169276150006477
1.091972783863581
1.1692747332665832
1.0919726360095239
1.1692733165018514
1.0919724881543098
1.169271899712276
1.0919723402979518
1.1692704828978455
1.0919721924404537
1.1692690660585559
1.0919720445818264
1.1692676491943996
1.0919718967220804
1.1692662323053717
1.091971748861221
1.1692648153914604
1.0919716009992588
1.1692633984526626
1.0919714531362026
1.1692619814889695
1.0919713052720605
1.1692605645003762
1.0919711574068405
1.1692591474868719
1.0919710095405526
1.169257730448452
1.0919708616732038
1.1692563133851097
1.0919707138048038
1.1692548962968363
1.091970565935362
1.1692534791836269
1.0919704180648855
1.1692520620454725
1.091970270193384
1.1692506448823672
1.0919701223208655
1.1692492276943032
1.0919699744473397
1.1692478104812745
1.0919698265728128
1.1692463932432717
1.0919696786972972
1.16924497598029
1.0919695308207986
1.1692435586923224
1.0919693829433272
1.

1.168658854802791
1.0919086248297825
1.1686574266128014
1.0919084772973875
1.1686559983948608
1.0919083297676584
1.1686545701489628
1.0919081822406036
1.168653141875099
1.091908034716233
1.1686517135732615
1.0919078871945558
1.168650285243444
1.0919077396755805
1.168648856885641
1.0919075921593147
1.1686474284998423
1.0919074446457706
1.1686460000860412
1.0919072971349535
1.1686445716442313
1.0919071496268737
1.1686431431744047
1.0919070021215411
1.1686417146765544
1.0919068546189636
1.1686402861506728
1.0919067071191504
1.168638857596753
1.0919065596221096
1.1686374290147865
1.0919064121278514
1.168636000404768
1.0919062646363837
1.168634571766689
1.0919061171477162
1.1686331431005426
1.0919059696618572
1.1686317144063199
1.091905822178817
1.1686302856840165
1.0919056746986018
1.1686288569336225
1.0919055272212224
1.1686274281551323
1.0919053797466893
1.1686259993485388
1.0919052322750085
1.1686245705138336
1.0919050848061913
1.1686231416510102
1.0919049373402436
1.16862171276006
1.09

In [ ]:
plt.title("Loss")
plt.xlabel("itoration")
plt.ylabel("Loss")
plt.plot(np.arange(1, len(train_varLoss) + 1), train_varLoss,color='red',alpha=0.5)
plt.plot(np.arange(1, len(test_varLoss) + 1), test_varLoss,color='blue',alpha=0.5)
plt.show()

In [ ]:
plt.title("Accuracy")
plt.xlabel("itoration")
plt.ylabel("accuracy")
plt.plot(np.arange(1, len(train_varAccuracy) + 1), train_varAccuracy,color='red',alpha=0.5)
plt.plot(np.arange(1, len(test_varAccuracy) + 1), test_varAccuracy,color='blue',alpha=0.5)
plt.show()

In [ ]:
plt.title("Time")
plt.xlabel("itoration")
plt.ylabel("Taka Times")
plt.plot(np.arange(1, len(train_varAccuracy) + 1), times,color='red',alpha=0.5)

plt.show()

In [ ]:
print(" | loss | accuracy |")
print("---------------------------------------")
print("training | %.2f | %.2f |" % (train_varLoss[-1], train_varAccuracy[-1]))
print("---------------------------------------")
print("validation | %.2f | %.2f |" % (test_varLoss[-1], test_varAccuracy[-1]))
print("---------------------------------------")